#### Three things
* Creating GUI
* Web scraping from cricbuzz to get all the score -> make http request and parse the web content
* display the live match update on our app

In [1]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
## importing necessary libraries or modules
from tkinter import *
from tkinter.ttk import Combobox
from PIL import ImageTk          # for adding background image to our gui
import requests                  # for making http request to a web page
from bs4 import BeautifulSoup    # for parsing html web pages
import re                        # for string matching (finding the team name from string)


In [19]:
class CricketScore:

    # creating a constructor
    # self is basically trying to represent the instance of that class
    def __init__(self, rootWindow):
        self.rootWindow=rootWindow
        self.rootWindow.title('LIVE CRICKET SCORE')
        self.rootWindow.geometry('640x359')
        self.bg=ImageTk.PhotoImage(file='bg.jpg')
        bg=Label(self.rootWindow,image=self.bg).place(x=0,y=0)

        # let's give label to my gui
        self.label=Label(self.rootWindow, text='Live Matches', font=("times new roman",30), compound="center").pack(padx=5,pady=5)
        
        # fetch the live match details and update it in GUI
        self.var=StringVar()
        self.matches=self.match_details()   # we dump all the match data
        self.data=[i for i in self.matches.keys()]
        self.cb=Combobox(self.rootWindow,values=self.data,width=50)
        self.cb.place(x=150,y=90)

        # button to check the details of match
        self.b1=Button(self.rootWindow,text='Check Results', font=('times new roman',15), command=self.show_match_details).place(x=230,y=120)


    # creating command for check score button
    def select(self):
        return self.cb.get()
    

    # function to scrap the cricbuzz
    def scrap(self):
        url='https://www.cricbuzz.com/'
        page=requests.get(url)    # fetch all the html content of that url
        # parse this response and fetch the relevant data
        # jisme hm interested h us section ka content nikalna h
        # we did using html div tags and each div have its own identifier
        soup=BeautifulSoup(page.content,'html.parser')
        results=soup.find(id='match_menu_container')   # list of scorecards
        scrap_results=results.find_all('li',class_='cb-match-card')
        # here we get all the scorecards
        return scrap_results
    

    # we use self in this to recognize it as the function belong to this class
    # in this function we will load all the matches details
    def match_details(self):

        # scrap the data
        details=self.scrap()

        # parse the relvant data 
        live_match={}
        # iterate over each score card and find the details of each match and store it
        for match in details:
            live_team_details={}
            summary=self.match_summary(match)
            if summary is not None:
                match_header=self.match_header(match).text
                teams=self.teams_name(match)
                scores=self.teams_score(match)
                live_team_details['summary']=summary.text
                live_team_details['match_header']=match_header
                live_team_details['scorecard']=scores[0]+' :: '+scores[1]
                live_match[teams[0]+' vs '+teams[1]]=live_team_details
        
        return live_match
    

    def match_summary(self,match):
        return match.find('div',class_='cb-mtch-crd-state')
    

    def match_header(self,match):
        return match.find('div',class_='cb-mtch-crd-hdr')
    

    def teams_name(self,match):
        names=[]
        team1=match.find('div',class_='cb-hmscg-bat-txt').text
        # find the index at which team1 name ends
        team1_index=re.search(r"\d",team1).start() if re.search(r"\d",team1) else len(team1)
        team2=match.find('div',class_='cb-hmscg-bwl-txt').text
        # find the index at which team2 name ends
        team2_index=re.search(r"\d",team2).start() if re.search(r"\d",team2) else len(team2)
        names.append(team1[:team1_index])
        names.append(team2[:team2_index])
        return names
    

    def teams_score(self,match):
        scores=[]
        team1=match.find('div',class_='cb-hmscg-bat-txt').text
        team2=match.find('div',class_='cb-hmscg-bwl-txt').text
        scores.append(team1)
        scores.append(team2)
        return scores


    # Show match details
    def show_match_details(self):

        # Building the match detail frame
        self.frame1=Frame(self.rootWindow,bg='#ADD8E6')
        self.frame1.place(x=20,y=180,width=600,height=150)

        # fetching details of the match
        x=self.matches[self.select()] 

        # displaying details of match
        Label(self.frame1, text=self.select() + " - " + x['match_header'],font=("times new roman",15,"bold"),bg="#ADD8E6",fg="red",bd=0).place(x=50, y=15)

        # Displaying details of the match
        Label(self.frame1,text="Score Details : ",font=("times new roman",10,"bold"),bg="#ADD8E6",fg="black",bd=0).place(x=10, y=40)
        Label(self.frame1,text=x['scorecard'],font=("times new roman", 10, "bold"),bg="#ADD8E6",fg="black",bd=0).place(x=20, y=60)
        Label(self.frame1,text="Summary : ",font=("times new roman", 10, "bold"),bg="#ADD8E6",fg="black",bd=0).place(x=10, y=100)
        Label(self.frame1,text=x['summary'],font=("times new roman", 10, "bold"),bg="#ADD8E6",fg="black",bd=0).place(x=20, y=120)


In [20]:
# main function to start the app
def main():
    # start the gui window

    # create an object of Tkinter class
    rootWindow=Tk()

    #fetch the cricket score from cricbuzz
    obj=CricketScore(rootWindow)

    # keep on displaying the gui window to fetch the updates
    rootWindow.mainloop()              # mainloop means run continuously


# execute the file
# if name=main is used to strictly check that the main function run from same file where it is define means it can't import from other file
if __name__=='__main__':
    main()